<a href="https://colab.research.google.com/github/mygoalistokillbill/mmds/blob/main/Colabs/CS246_Colab_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS246 - Colab 2
## Frequent Pattern Mining in Spark

### Setup

Let's setup Spark on your Colab environment.  Run the cell below!

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 212.4 MB 69 kB/s 
     |████████████████████████████████| 198 kB 54.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=35a0db445c6daf747cd09ce51d954b0ab7ff86b5d6af7815f7ea59afa4f3e723
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 40 not upgraded.
Need to get 36.5 MB of archives.
After t

Now we authenticate a Google Drive client to download the file we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
id='1dhi1F78ssqR8gE6U-AgB80ZW7V_9snX4'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('products.csv')

id='1KZBNEaIyMTcsRV817us6uLZgm-Mii8oU'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('order_products__train.csv')

If you executed the cells above, you should be able to see the dataset we will need for this Colab under the "Files" tab on the left panel.

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

Let's initialize the Spark context.

In [5]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

You can easily check the current version and get the link of the web interface. In the Spark UI, you can monitor the progress of your job and debug the performance bottlenecks (if your Colab is running with a **local runtime**).

In [6]:
spark

If you are running this Colab on the Google hosted runtime, the cell below will create a *ngrok* tunnel which will allow you to still check the Spark UI.

In [7]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2021-08-17 09:23:51--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.44.51.219, 52.202.35.83, 34.226.109.249, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.44.51.219|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  16.7MB/s    in 0.8s    

2021-08-17 09:23:52 (16.7 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
Traceback (most recent call last):
  File "<string>", line 1, in <module>
IndexError: list index out of range


### Your task

If you run successfully the setup stage, you are ready to work with the **3 Million Instacart Orders** dataset. In case you want to read more about it, check the [official Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) about it, a concise [schema description](https://gist.github.com/jeremystan/c3b39d947d9b88b3ccff3147dbcf6c6b) of the dataset, and the [download page](https://www.instacart.com/datasets/grocery-shopping-2017).

In this Colab, we will be working only with a small training dataset (~131K orders) to perform fast Frequent Pattern Mining with the FP-Growth algorithm.

In [8]:
products = spark.read.csv('products.csv', header=True, inferSchema=True)
orders = spark.read.csv('order_products__train.csv', header=True, inferSchema=True)

In [9]:
products.printSchema()

root
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- aisle_id: string (nullable = true)
 |-- department_id: string (nullable = true)



In [10]:
orders.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- add_to_cart_order: integer (nullable = true)
 |-- reordered: integer (nullable = true)



Use the Spark Dataframe API to join 'products' and 'orders', so that you will be able to see the product names in each transaction (and not only their ids).  Then, group by the orders by 'order_id' to obtain one row per basket (i.e., set of products purchased together by one customer). 

In [21]:
# YOUR CODE HERE
orders_with_product_name = orders.join(products, orders.product_id == products.product_id)
orders_with_product_name.printSchema()
buskets=orders_with_product_name.groupBy('order_id').agg(collect_list("product_name").alias("names"))
buskets.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- add_to_cart_order: integer (nullable = true)
 |-- reordered: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- aisle_id: string (nullable = true)
 |-- department_id: string (nullable = true)

root
 |-- order_id: integer (nullable = true)
 |-- names: array (nullable = false)
 |    |-- element: string (containsNull = false)



In this Colab we will explore [MLlib](https://spark.apache.org/mllib/), Apache Spark's scalable machine learning library. Specifically, you can use its implementation of the [FP-Growth](https://spark.apache.org/docs/latest/ml-frequent-pattern-mining.html#fp-growth) algorithm to perform efficiently Frequent Pattern Mining in Spark.
Use the Python example in the documentation, and train a model with 

```minSupport=0.01``` and ```minConfidence=0.5```



In [24]:
# YOUR CODE HERE
from pyspark.ml.fpm import FPGrowth

fpGrowth = FPGrowth(itemsCol="names", minSupport=0.1, minConfidence=0.5)
model = fpGrowth.fit(buskets)

+--------------------+-----+
|               items| freq|
+--------------------+-----+
|            [Banana]|18726|
|[Bag of Organic B...|15480|
+--------------------+-----+

+----------+----------+----------+----+-------+
|antecedent|consequent|confidence|lift|support|
+----------+----------+----------+----+-------+
+----------+----------+----------+----+-------+

+--------+--------------------+----------+
|order_id|               names|prediction|
+--------+--------------------+----------+
|    1342|[Bag of Organic B...|        []|
|    1591|[Cracked Wheat, O...|        []|
|    4519|[Beet Apple Carro...|        []|
|    4935|             [Vodka]|        []|
|    6357|[Fresh Mozzarella...|        []|
|   10362|[Crinkle Cut Fren...|        []|
|   19204|[Lemon Lime Thirs...|        []|
|   29601|[Toasted Coconut ...|        []|
|   31035|[Bag of Organic B...|        []|
|   40011|[Sea Salt Macadam...|        []|
|   46266|[Organic Uncured ...|        []|
|   51607|[Major Dickason's...

Compute how many frequent itemsets and association rules were generated by running FP-growth.


In [25]:
# YOUR CODE HERE
# Display frequent itemsets.
model.freqItemsets.show()

# Display generated association rules.
model.associationRules.show()

# transform examines the input items against all the association rules and summarize the
# consequents as prediction
model.transform(buskets).show()

+--------------------+-----+
|               items| freq|
+--------------------+-----+
|            [Banana]|18726|
|[Bag of Organic B...|15480|
+--------------------+-----+

+----------+----------+----------+----+-------+
|antecedent|consequent|confidence|lift|support|
+----------+----------+----------+----+-------+
+----------+----------+----------+----+-------+

+--------+--------------------+----------+
|order_id|               names|prediction|
+--------+--------------------+----------+
|    1342|[Bag of Organic B...|        []|
|    1591|[Cracked Wheat, O...|        []|
|    4519|[Beet Apple Carro...|        []|
|    4935|             [Vodka]|        []|
|    6357|[Fresh Mozzarella...|        []|
|   10362|[Crinkle Cut Fren...|        []|
|   19204|[Lemon Lime Thirs...|        []|
|   29601|[Toasted Coconut ...|        []|
|   31035|[Bag of Organic B...|        []|
|   40011|[Sea Salt Macadam...|        []|
|   46266|[Organic Uncured ...|        []|
|   51607|[Major Dickason's...

Now retrain the FP-growth model changing only 
```minsupport=0.001``` 
and compute how many frequent itemsets and association rules were generated.


In [33]:
# YOUR CODE HERE
fpGrowth = FPGrowth(itemsCol="names", minSupport=0.001, minConfidence=0.5)
model = fpGrowth.fit(buskets)

# Display frequent itemsets.
model.freqItemsets.show(truncate=False)

# Display generated association rules.
model.associationRules.show(truncate=False)

# transform examines the input items against all the association rules and summarize the
# consequents as prediction
model.transform(buskets).filter(size("prediction") > 0).show()

+-----------------------------------------------------------------+-----+
|items                                                            |freq |
+-----------------------------------------------------------------+-----+
|[Organic Tomato Basil Pasta Sauce]                               |772  |
|[Organic Tomato Basil Pasta Sauce, Bag of Organic Bananas]       |175  |
|[Organic Tomato Basil Pasta Sauce, Organic Baby Spinach]         |144  |
|[Organic Tomato Basil Pasta Sauce, Banana]                       |179  |
|[Organic Spinach Bunch]                                          |475  |
|[Whole Milk Ricotta Cheese]                                      |347  |
|[Medium Salsa Roja]                                              |275  |
|[Ground Buffalo]                                                 |231  |
|[Tonic Water]                                                    |194  |
|[Original Coconut Milk Beverage]                                 |173  |
|[Low-Fat Strawberry Banana on the Bot